<a href="https://colab.research.google.com/github/vloneonme/trew/blob/main/san_lr5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практическая работа: Объектно-ориентированное программирование

## Вариант 3: Управление вычислительными задачами в HPC-среде

Данная работа демонстрирует основные концепции ООП на примере симуляции системы очередей вычислительных задач в высокопроизводительной вычислительной среде (HPC):

- Инкапсуляция (@property и сеттеры)
- Наследование (MPITask и GPUTask наследуют от ComputeTask)
- Полиморфизм (переопределение execute(), __str__)
- Магические методы (__init__, __str__, __repr__)
- Композиция (ComputeQueue содержит список задач)
- Агрегация (Scheduler управляет несколькими очередями)

In [13]:
# Импортируем необходимые модули
from typing import List, Optional
import datetime
print("Библиотеки импортированы")

Библиотеки импортированы


In [14]:
# === Определение классов ===

class ComputeTask:
    """
    Базовый класс для вычислительной задачи.
    Демонстрирует инкапсуляцию.
    """
    def __init__(self, task_id: str, required_cores: int, required_memory_gb: float):
        self._task_id = task_id
        self._required_cores = required_cores
        self._required_memory_gb = required_memory_gb
        self._status = "pending"  # pending, running, completed
        self._start_time: Optional[datetime.datetime] = None
        self._end_time: Optional[datetime.datetime] = None

    @property
    def task_id(self) -> str:
        return self._task_id

    @property
    def required_cores(self) -> int:
        return self._required_cores

    @property
    def required_memory_gb(self) -> float:
        return self._required_memory_gb

    @property
    def status(self) -> str:
        return self._status

    @status.setter
    def status(self, value: str):
        if value not in ["pending", "running", "completed"]:
            raise ValueError("Статус должен быть: pending, running или completed")
        self._status = value

    def execute(self) -> None:
        """Базовое выполнение задачи"""
        if self._status == "pending":
            self._status = "running"
            self._start_time = datetime.datetime.now()
            print(f"Задача {self.task_id} запущена.")
            # Симуляция выполнения
            self._status = "completed"
            self._end_time = datetime.datetime.now()
            print(f"Задача {self.task_id} завершена.")
        else:
            print(f"Задача {self.task_id} уже в статусе {self._status}")

    def __str__(self) -> str:
        return (f"Задача {self.task_id} | Ядер: {self.required_cores} | "
                f"Память: {self.required_memory_gb} GB | Статус: {self.status}")

    def __repr__(self) -> str:
        return (f"ComputeTask(task_id='{self.task_id}', cores={self.required_cores}, "
                f"memory={self.required_memory_gb}GB, status='{self.status}')")


class MPITask(ComputeTask):
    """MPI-задача — наследование и полиморфизм"""
    def __init__(self, task_id: str, required_cores: int, required_memory_gb: float, nodes_count: int):
        super().__init__(task_id, required_cores, required_memory_gb)
        self._nodes_count = nodes_count

    @property
    def nodes_count(self) -> int:
        return self._nodes_count

    def execute(self) -> None:
        if self.status == "pending":
            print(f"MPI-задача {self.task_id} запущена на {self.nodes_count} узлах.")
            super().execute()
            print(f"MPI-задача {self.task_id} успешно распределена.")
        else:
            print(f"MPI-задача {self.task_id} уже выполняется/завершена.")

    def __str__(self) -> str:
        return super().__str__() + f" | Узлы: {self.nodes_count} (MPI)"


class GPUTask(ComputeTask):
    """GPU-задача — наследование и полиморфизм"""
    def __init__(self, task_id: str, required_cores: int, required_memory_gb: float, gpu_type: str):
        super().__init__(task_id, required_cores, required_memory_gb)
        self._gpu_type = gpu_type

    @property
    def gpu_type(self) -> str:
        return self._gpu_type

    def execute(self) -> None:
        if self.status == "pending":
            print(f"GPU-задача {self.task_id} запущена на GPU {self.gpu_type}.")
            super().execute()
            print(f"GPU-задача {self.task_id} завершена с ускорением.")
        else:
            print(f"GPU-задача {self.task_id} уже выполняется/завершена.")

    def __str__(self) -> str:
        return super().__str__() + f" | GPU: {self.gpu_type}"


class ComputeQueue:
    """Очередь задач — демонстрирует композицию"""
    def __init__(self, name: str, max_cores: int, max_memory: float):
        self.name = name
        self.max_cores = max_cores
        self.max_memory = max_memory
        self.queue: List[ComputeTask] = []

    def add_task(self, task: ComputeTask) -> bool:
        current_cores = sum(t.required_cores for t in self.queue if t.status == "running")
        current_memory = sum(t.required_memory_gb for t in self.queue if t.status == "running")

        if (current_cores + task.required_cores <= self.max_cores and
                current_memory + task.required_memory_gb <= self.max_memory):
            self.queue.append(task)
            print(f"Задача {task.task_id} добавлена в очередь '{self.name}'")
            return True
        else:
            print(f"Недостаточно ресурсов для задачи {task.task_id} в очереди '{self.name}'")
            return False

    def run_next(self) -> None:
        for task in self.queue:
            if task.status == "pending":
                running_cores = sum(t.required_cores for t in self.queue if t.status == "running")
                running_memory = sum(t.required_memory_gb for t in self.queue if t.status == "running")
                if (running_cores + task.required_cores <= self.max_cores and
                        running_memory + task.required_memory_gb <= self.max_memory):
                    task.execute()
                    return
        print(f"Нет доступных задач для запуска в очереди '{self.name}'")

    def show_queue(self) -> None:
        print(f"\n=== Очередь: {self.name} ===")
        print(f"Лимит: {self.max_cores} ядер, {self.max_memory} GB памяти")
        if not self.queue:
            print("Очередь пуста")
        else:
            for task in self.queue:
                print(task)
        print("=" * 30)


# Дополнительное задание
class Scheduler:
    """Планировщик — агрегация нескольких очередей"""
    def __init__(self, name: str):
        self.name = name
        self.queues: List[ComputeQueue] = []

    def add_queue(self, queue: ComputeQueue):
        self.queues.append(queue)
        print(f"Очередь '{queue.name}' добавлена в планировщик")

    def run_all(self):
        print(f"\n--- Цикл планировщика {self.name} ---")
        for queue in self.queues:
            queue.run_next()

    def show_all(self):
        print(f"\n=== Все очереди в планировщике {self.name} ===")
        for queue in self.queues:
            queue.show_queue()

## Демонстрация работы системы

In [15]:
# Создаём задачи разных типов
task1 = ComputeTask("T001", required_cores=8, required_memory_gb=16)
task2 = MPITask("T002", required_cores=64, required_memory_gb=128, nodes_count=8)
task3 = GPUTask("T003", required_cores=4, required_memory_gb=32, gpu_type="A100")
task4 = ComputeTask("T004", required_cores=16, required_memory_gb=32)
task5 = GPUTask("T005", required_cores=8, required_memory_gb=64, gpu_type="V100")

print("Примеры задач:")
print(task1)
print(task2)
print(task3)
print()

Примеры задач:
Задача T001 | Ядер: 8 | Память: 16 GB | Статус: pending
Задача T002 | Ядер: 64 | Память: 128 GB | Статус: pending | Узлы: 8 (MPI)
Задача T003 | Ядер: 4 | Память: 32 GB | Статус: pending | GPU: A100



In [16]:
# Создаём очереди
cpu_queue = ComputeQueue("CPU-Queue", max_cores=128, max_memory=256)
gpu_queue = ComputeQueue("GPU-Queue", max_cores=32, max_memory=128)

# Добавляем задачи
cpu_queue.add_task(task1)
cpu_queue.add_task(task4)
cpu_queue.add_task(task2)  # MPI-задача

gpu_queue.add_task(task3)
gpu_queue.add_task(task5)
gpu_queue.add_task(task2)  # Не поместится — проверка ресурсов

# Показываем состояние
cpu_queue.show_queue()
gpu_queue.show_queue()

Задача T001 добавлена в очередь 'CPU-Queue'
Задача T004 добавлена в очередь 'CPU-Queue'
Задача T002 добавлена в очередь 'CPU-Queue'
Задача T003 добавлена в очередь 'GPU-Queue'
Задача T005 добавлена в очередь 'GPU-Queue'
Недостаточно ресурсов для задачи T002 в очереди 'GPU-Queue'

=== Очередь: CPU-Queue ===
Лимит: 128 ядер, 256 GB памяти
Задача T001 | Ядер: 8 | Память: 16 GB | Статус: pending
Задача T004 | Ядер: 16 | Память: 32 GB | Статус: pending
Задача T002 | Ядер: 64 | Память: 128 GB | Статус: pending | Узлы: 8 (MPI)

=== Очередь: GPU-Queue ===
Лимит: 32 ядер, 128 GB памяти
Задача T003 | Ядер: 4 | Память: 32 GB | Статус: pending | GPU: A100
Задача T005 | Ядер: 8 | Память: 64 GB | Статус: pending | GPU: V100


In [17]:
# Запуск задач по очереди
print("Запуск задач:")
cpu_queue.run_next()  # T001
cpu_queue.run_next()  # T004 (если хватает ресурсов)
gpu_queue.run_next()  # T003

cpu_queue.show_queue()
gpu_queue.show_queue()

Запуск задач:
Задача T001 запущена.
Задача T001 завершена.
Задача T004 запущена.
Задача T004 завершена.
GPU-задача T003 запущена на GPU A100.
Задача T003 запущена.
Задача T003 завершена.
GPU-задача T003 завершена с ускорением.

=== Очередь: CPU-Queue ===
Лимит: 128 ядер, 256 GB памяти
Задача T001 | Ядер: 8 | Память: 16 GB | Статус: completed
Задача T004 | Ядер: 16 | Память: 32 GB | Статус: completed
Задача T002 | Ядер: 64 | Память: 128 GB | Статус: pending | Узлы: 8 (MPI)

=== Очередь: GPU-Queue ===
Лимит: 32 ядер, 128 GB памяти
Задача T003 | Ядер: 4 | Память: 32 GB | Статус: completed | GPU: A100
Задача T005 | Ядер: 8 | Память: 64 GB | Статус: pending | GPU: V100


In [18]:
# Демонстрация полиморфизма
print("Полиморфизм метода execute():")
tasks = [task1, task2, task3, task4, task5]
for t in tasks:
    if t.status == "pending":
        t.execute()

Полиморфизм метода execute():
MPI-задача T002 запущена на 8 узлах.
Задача T002 запущена.
Задача T002 завершена.
MPI-задача T002 успешно распределена.
GPU-задача T005 запущена на GPU V100.
Задача T005 запущена.
Задача T005 завершена.
GPU-задача T005 завершена с ускорением.


In [19]:
# Дополнительное задание: Scheduler
scheduler = Scheduler("Главный HPC Планировщик")
scheduler.add_queue(cpu_queue)
scheduler.add_queue(gpu_queue)

scheduler.show_all()

print("Автоматическое выполнение через планировщик:")
scheduler.run_all()
scheduler.run_all()

scheduler.show_all()

Очередь 'CPU-Queue' добавлена в планировщик
Очередь 'GPU-Queue' добавлена в планировщик

=== Все очереди в планировщике Главный HPC Планировщик ===

=== Очередь: CPU-Queue ===
Лимит: 128 ядер, 256 GB памяти
Задача T001 | Ядер: 8 | Память: 16 GB | Статус: completed
Задача T004 | Ядер: 16 | Память: 32 GB | Статус: completed
Задача T002 | Ядер: 64 | Память: 128 GB | Статус: completed | Узлы: 8 (MPI)

=== Очередь: GPU-Queue ===
Лимит: 32 ядер, 128 GB памяти
Задача T003 | Ядер: 4 | Память: 32 GB | Статус: completed | GPU: A100
Задача T005 | Ядер: 8 | Память: 64 GB | Статус: completed | GPU: V100
Автоматическое выполнение через планировщик:

--- Цикл планировщика Главный HPC Планировщик ---
Нет доступных задач для запуска в очереди 'CPU-Queue'
Нет доступных задач для запуска в очереди 'GPU-Queue'

--- Цикл планировщика Главный HPC Планировщик ---
Нет доступных задач для запуска в очереди 'CPU-Queue'
Нет доступных задач для запуска в очереди 'GPU-Queue'

=== Все очереди в планировщике Главный

## Выводы

### Реализованные концепции ООП:
1. **Инкапсуляция** — через @property и защищённые атрибуты (_status и др.)
2. **Наследование** — MPITask и GPUTask наследуют от ComputeTask
3. **Полиморфизм** — переопределение execute() и __str__() в дочерних классах
4. **Магические методы** — __init__, __str__, __repr__
5. **Композиция** — ComputeQueue содержит список объектов ComputeTask
6. **Агрегация** — Scheduler содержит несколько очередей ComputeQueue

### Функциональность:
- Симуляция распределения ресурсов в HPC-кластере
- Проверка доступности ядер и памяти перед добавлением/запуском задач
- Разное поведение задач в зависимости от типа (CPU, MPI, GPU)
- Глобальное управление через планировщик

Работа полностью соответствует требованиям задания и готова к демонстрации в Google Colab.